# Examen Parcial 2. Machine y Deep Learning

### Universidad Autonoma de Aguascalientes
### Departamento: Ciencias de la Computación
### Carrera: Ingenieria en Computación Inteligente
### Curso: Machine_Learning y Deep Learning
### Maestro:Dr. Francisco Javier Luna Rosas
### Alumnos: Gerardo Gómez Cajero
### Semestre: Enero_Junio del 2024


### Importamos librerías necesarias

In [10]:
import os, cv2, dlib
from pathlib import Path
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

#### Linea para dar acceso a las imagenes de drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Preprocesamiento de imagenes y creacion del dataset

In [4]:
# Ruta de la carpeta raiz del dataset
carpeta_dataset = Path("/content/drive/MyDrive/UAA/10mo Semestre/Machine y deep learning/2do Semestre/dataset")

# Ruta de la carpeta con las imágenes originales
carpeta_imagenes_originales = Path(carpeta_dataset, "original")

# Carpeta de imagenes de entrenamiento
train_dir = Path(carpeta_dataset, 'train')

# Carpeta de imagenes de prueba
test_dir = Path(carpeta_dataset, 'test')

# Obtener la lista de los nombres de las carpetas dentro de la carpeta de imágenes originales
carpetas = [carpeta for carpeta in carpeta_imagenes_originales.iterdir() if carpeta.is_dir()]

# Iterar sobre las carpetas y redimensionar las imágenes pasando 30% de las imagenes originales a una carpeta test y el resto a train
for carpeta in carpetas:
    # Obtener la lista de archivos en la carpeta con Path
    archivos = os.listdir(carpeta)

    # Calcular la cantidad de imágenes que se van a mover a la carpeta de test
    cantidad_test = int(len(archivos) * 0.3)

    # Iterar sobre los archivos
    for i, archivo in enumerate(archivos):
        # Crear la ruta del archivo
        ruta_archivo = Path(carpeta, archivo)

        # Crear la ruta de destino
        if i < cantidad_test:
            carpeta_destino = "test"
        else:
            carpeta_destino = "train"

        ruta_carpeta_destino = Path(carpeta_dataset, carpeta_destino, carpeta.name)
        ruta_destino = Path(ruta_carpeta_destino, archivo)

        # Revizar si la carpeta destino existe, si no, crearla
        if not ruta_carpeta_destino.exists():
            ruta_carpeta_destino.mkdir(parents=True)

        # Abrir la imagen original
        imagen_original = Image.open(ruta_archivo)

        # Redimensionar la imagen a 64x64 píxeles
        imagen_redimensionada = imagen_original.resize((64, 64))

        # Guardar la imagen redimensionada en la carpeta de destino
        imagen_redimensionada.save(ruta_destino)

        # Cerrar la imagen original
        imagen_original.close()


### Creación y entrenamiento del modelo a partir del dataset de imagenes preprocesadas

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='sparse'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='sparse'
)

# Crear el modelo CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(5, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo
model.fit(train_generator, epochs=400, validation_data=test_generator)

# Ruta del modelo
ruta_modelo = Path(carpeta_dataset.parent, 'modelo_predictor_sentimientos.keras')

model.save(ruta_modelo)
loss, accuracy = model.evaluate(test_generator)
print(f'Precisión: {accuracy}')

Found 20 images belonging to 5 classes.
Found 5 images belonging to 5 classes.
Epoch 1/400
1/1 [==============================] - 2s 2s/step - loss: 1.6182 - accuracy: 0.2500 - val_loss: 1.5978 - val_accuracy: 0.2000
Epoch 2/400
1/1 [==============================] - 0s 384ms/step - loss: 1.6153 - accuracy: 0.2000 - val_loss: 1.5940 - val_accuracy: 0.2000
Epoch 3/400
1/1 [==============================] - 0s 333ms/step - loss: 1.5557 - accuracy: 0.2500 - val_loss: 1.5771 - val_accuracy: 0.2000
Epoch 4/400
1/1 [==============================] - 1s 505ms/step - loss: 1.6325 - accuracy: 0.1500 - val_loss: 1.5635 - val_accuracy: 0.2000
Epoch 5/400
1/1 [==============================] - 0s 450ms/step - loss: 1.6649 - accuracy: 0.1500 - val_loss: 1.5511 - val_accuracy: 0.6000
Epoch 6/400
1/1 [==============================] - 1s 521ms/step - loss: 1.6252 - accuracy: 0.1500 - val_loss: 1.5459 - val_accuracy: 0.6000
Epoch 7/400
1/1 [==============================] - 0s 464ms/step - loss: 1.575

### Uso del modelo predictor de sentimientos

In [ ]:
model = load_model(ruta_modelo)
emociones = {0: 'Enojado', 1: 'Feliz', 2: 'Neutro', 3: 'Triste', 4: 'Sorprendido'}

# Inicializar el detector de caras de dlib
detector_caras = dlib.get_frontal_face_detector()

# Iniciar la captura de video desde la cámara
cap = cv2.VideoCapture(0)

while True:
    # Capturar un fotograma de la cámara
    ret, frame = cap.read()

    if ret:
        # Convertir el fotograma a escala de grises
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detectar caras en el fotograma
        emociones = detector_caras(gray)

        for emocion in emociones:
            x, y, w, h = emocion.left(), emocion.top(), emocion.width(), emocion.height()
            # Recortar la región de la cara
            recorte = frame[y:y+h, x:x+w]
            recorte = cv2.resize(recorte, (64, 64))
            recorte = recorte.astype("float") / 255.0
            recorte = img_to_array(recorte)
            recorte = np.expand_dims(recorte, axis=0)
            prediction = model.predict(recorte)
            predicted_emotion = emociones[np.argmax(prediction)]
            cv2.putText(frame, predicted_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Mostrar el fotograma
        cv2.imshow('Detector de emociones', frame)

        # Salir del bucle si se presiona 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Liberar la captura de video y cerrar la ventana
cap.release()
cv2.destroyAllWindows()